## Sanbox - Py36

In [2]:
import pandas as pd
import numpy as np
import re
import calendar 
import pydoc
import types
import xlrd
import pickle
import random

import datetime as dt

import sys as sys
import os as os
import operator as operator
from collections import Counter

print ("Running system under path: {}\n".format(sys.prefix))

print ("System version: {}".format(sys.version))
print ("Pandas version: {}".format(pd.__version__))
print ("Numpy version: {}".format(np.__version__))
print ("\n")

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

Running system under path: C:\Users\patblo\anaconda3

System version: 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version: 0.24.2
Numpy version: 1.17.0





## Spear

In [3]:
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble.partial_dependence import plot_partial_dependence


from sklearn.model_selection import GridSearchCV, learning_curve, ShuffleSplit, RandomizedSearchCV, train_test_split, cross_val_score, validation_curve, KFold

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder

from sklearn.feature_selection import RFE,SelectKBest

from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, accuracy_score, roc_auc_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import _name_estimators

from sklearn.externals import joblib
from sklearn.externals import six

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.base import clone
from sklearn.base import TransformerMixin



C:\Users\patblo\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\patblo\anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## Options

In [4]:
# Widen notebook
from IPython.core.display import HTML, display
HTML("<style>.container { width:85% !important; }</style>")

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.expand_frame_repr', False)
pd.options.mode.chained_assignment = None

## Functions

In [6]:
from dateutil import relativedelta as rtd
from datetime import datetime, timedelta, date

#--------------------------------- 
# TIME DIMENSIONS
#---------------------------------
def f_this_ym():
    return dt.date.today().strftime('%Y%m')

def f_last_ym():
    return (dt.date.today() - rtd.relativedelta(months=+1)).strftime("%Y%m")

def f_last_ymd_01():
    return (dt.date.today() - rtd.relativedelta(months=+1)  - rtd.relativedelta(day = 1)).strftime("%Y%m%d")

def f_t_now():
    return dt.datetime.today().strftime("%H:%M:%S")

def f_d_now():
    return dt.date.today().strftime("%Y-%m-%d")

def f_dt_now():
    return dt.datetime.today().strftime("%Y-%m-%d %H:%M:%S")


---
# Function simplicity. 


Same output, wastly different code (performance, and readability). Simplicity  matters! 

---

In [7]:
def f_apply_bin_reduction_method(exec_f, indata, list_reduction_method, sclr_pre_post_mode='pre'):

    """
    This function itterates over a dataframe which has monotonically decreasing values, and bins the rows into larger groups based on a proportional reduction 
    coming from f_bin_reduction_method function. 
    """

    if exec_f:

        import pandas as pd
        import numpy as np
        import time as time

        df_temp=df_temp_pre.copy()
        sclr_pre_post_mode='pre'

        list_hld_df_idx=list()
        list_hld_df=list()

        print ("Datetime is:  {}. Nr of rows going in is: {}\n".format(f_dt_now(),len(df_temp)))
        tick=time.time()


        #-------------------------------
        # Prepp data before logic
        #-------------------------------

        # The values are increasing, i.e. we are monotonically increasing over distribution, pre mode point
        if sclr_pre_post_mode=='pre':
            print ("Pre central mode point\n")

            # Highest to lowest sort, and reset index!
            df_temp=df_temp.sort_index(ascending=False)

        # We are post mode point
        else:
            print ("Post central mode point\n")


        #-----------------------------------
        # Itterate through reduction rules
        #-----------------------------------
        for idx_reduction_lvl, reduction_lvl in enumerate(list_reduction_method):

            print ("Processing reduction lvl: {} being a proportional shift of: {} from central mode point.".format(idx_reduction_lvl, reduction_lvl))

            #-------------------------------------------------------
            # Reduction lvl 1: Whole DataFrame is processed
            #-------------------------------------------------------
            if idx_reduction_lvl==0:

                # Temp variables (cum sum % and cum sum observations)
                df_temp['tmp_cumsum_%']=np.cumsum(df_temp['count_obs_%_tot'])
                df_temp['tmp_cumsum_obs']=np.cumsum(df_temp['count_obs'])

                # what group are we processing?
                df_temp['grp_reduction_lvl']=idx_reduction_lvl + 1

                # Flag rows
                df_temp['f_reduced_grp']=np.where(df_temp['tmp_cumsum_%']<=reduction_lvl
                                                  ,1
                                                  ,0
                                                 )

                # Append subset result from flag
                list_hld_df.append(df_temp[df_temp['f_reduced_grp']==1])
                list_hld_df_idx.extend(df_temp[df_temp['f_reduced_grp']==1].index.tolist())

            #--------------------------------------------------------------------------------------------------
            # Reduction lvl 1+i: Subset DataFrame is processed (removing previous processed index rows)
            #--------------------------------------------------------------------------------------------------
            else:

                # Remove index rows we already have processed
                df_temp_sub=df_temp[~df_temp.index.isin(list_hld_df_idx)]

                # Temp variables (cum sum % and cum sum observations)
                df_temp_sub['tmp_cumsum_%']=np.cumsum(df_temp_sub['count_obs_%_tot'])
                df_temp_sub['tmp_cumsum_obs']=np.cumsum(df_temp_sub['count_obs'])

                # what group are we processing?
                df_temp_sub['grp_reduction_lvl']=idx_reduction_lvl + 1

                # Flag rows
                df_temp_sub['f_reduced_grp']=np.where(df_temp_sub['tmp_cumsum_%']<=reduction_lvl
                                                  ,1
                                                  ,0)

                # Append subset result to holder
                list_hld_df.append(df_temp_sub[df_temp_sub['f_reduced_grp']==1])
                list_hld_df_idx.extend(df_temp_sub[df_temp_sub['f_reduced_grp']==1].index.tolist())

        #-------------------------------------
        # Append to a common DataFrame
        #-------------------------------------
        df_temp=pd.concat(list_hld_df, axis=0)

        # Propagate bin edge, given pre or post mode point, throughout whole dataframe
        if sclr_pre_post_mode=='pre':
            df_temp['bin_edge_grp_lower'] = df_temp[['grp_reduction_lvl','bin_upper_edge']].groupby(['grp_reduction_lvl']).transform('first')

        elif sclr_pre_post_mode == 'post':
            df_temp['bin_edge_grp_lower'] = df_temp[['grp_reduction_lvl','bin_lower_edge']].groupby(['grp_reduction_lvl']).transform('first')

            
        print ("\nTotal elapsed time is: {}s. Nr of rows going out is: {}".format(round(time.time()-tick), len(df_temp)))
        
        return df_temp

    else:
        print ("No execution of function, ending...")


In [8]:
def f_apply_bin_reduction_method(exec_f, indata, list_reduction_method, sclr_pre_post_mode='pre'):

    """
    This function itterates over a dataframe which has monotonically decreasing values, and bins the rows into larger groups based on a proportional reduction 
    coming from f_bin_reduction_method function. 
    """

    # Modules
    import pandas as pd
    import numpy as np
    import time as time

    # Inputs
    df_temp=indata.copy()
    list_hld_reduction_method=list_reduction_method
    
    # time stamp
    try:
        print ("Datetime now is: {}. Nr of rows going in is: {}\n".format(f_dt_now(), len(df_temp)))
    
    except:
        print ("No datetime function loaded")
    
    # Start counting time
    tick=time.time()

    #-------------------------------
    # Prepp data before logic
    #-------------------------------
           
    # The values are increasing, i.e. we are monotonically increasing over distribution, pre mode point
    if sclr_pre_post_mode=='pre':
    
        print ("Pre central mode point\n")
        bool_pre=True
                            
        # Highest to lowest sort, and make a copy so things stick! 
        df_temp=df_temp.sort_index(ascending=False)
        df_temp_copy=df_temp.copy()
        
        # Reset index for itteration logic to work
        df_temp.reset_index(drop=True, inplace=True)
        df_temp_copy.reset_index(drop=True, inplace=True)
        
    # We are post mode point
    else:
        
        print ("post central mode point\n")
        bool_pre=False
        
        # Make a copy so things stick! 
        df_temp_copy=df_temp.copy()
        
        # Reset index for itteration logic to work
        df_temp.reset_index(drop=True, inplace=True)
        df_temp_copy.reset_index(drop=True, inplace=True)

        
    #-------------------------------
    # Apply itteration logic
    #-------------------------------

    
    # Parameters and data holders
    sclr_sum_prct_tot = 0
    sclr_sum_count_tot = 0
    sclr_reduction_lvl = 0
    sclr_bin_edge=0
    list_hld_df_data_reduced = list()
    
    # Itterate over data rows with mode point as reference. We write to the copy DataFrame to avoid things getting lost! 
    for df_row in df_temp.iterrows():

        # Add together % of total, and count, given row and adherent bin
        sclr_sum_prct_tot += df_row[1]['count_obs_%_tot']    
        sclr_sum_count_tot += df_row[1]['count_obs']    

        # Set group reduction lvl
        df_temp_copy.loc[df_row[0], 'grp_reduction_lvl']=sclr_reduction_lvl

        # Set group reduction lvl prct of total sum and count of total sum
        df_temp_copy.loc[df_row[0], 'sum_prct_tot']=sclr_sum_prct_tot
        df_temp_copy.loc[df_row[0], 'sum_count_tot']=sclr_sum_count_tot

        # Start value: This picks up first row if we dont surpass the reduction level
        if df_row[0]==0:
            
            # Initialize bin edges: We need to keep track of start and end points for each new bin, and also if the values are monotonic increase/decrease
            if (bool_pre==True):
                
                # Start point: Set group reduction lvl count of total sum
                sclr_bin_edge=df_temp_copy.loc[df_row[0], 'bin_upper_edge']
                df_temp_copy.loc[df_row[0], 'bin_edge_lag1']=sclr_bin_edge

            elif (bool_pre==False):

                # Start point: Set group reduction lvl count of total sum
                sclr_bin_edge=df_temp_copy.loc[df_row[0], 'bin_upper_edge']                
                df_temp_copy.loc[df_row[0], 'bin_edge_lag1']=sclr_bin_edge
            
            # Push out start point, and lower/upper bin edge depending on monotonic increase/decrease
            print ("FIRST ROW: {} \n".format(df_row[0]))
            list_hld_df_data_reduced.append(df_temp_copy[df_temp_copy.index==df_row[0]])

            
        # Populate bins edges
        df_temp_copy.loc[df_row[0], 'bin_edge_lag1']=sclr_bin_edge

    
        # If we hit a given reduction level, mark the row, set counter to 0 and start with next reduction level. This will also overvrite first row above if we get a hit down here 
        if sclr_sum_prct_tot > list_hld_reduction_method[sclr_reduction_lvl]:

            # Set group reduction lvl
            df_temp_copy.loc[df_row[0], 'grp_reduction_lvl']=sclr_reduction_lvl

            # Set group reduction lvl prct of total sum and count of total sum
            df_temp_copy.loc[df_row[0], 'sum_prct_tot']=sclr_sum_prct_tot
            df_temp_copy.loc[df_row[0], 'sum_count_tot']=sclr_sum_count_tot
            
            # Update bin edges: We need to keep track of start and end points, and also if the values are monotonic increasing or decreasing
            if (bool_pre==True) & (df_row[0] > 0):
                
                # New update edge, looking back one row
                sclr_bin_edge=df_temp_copy.loc[(df_row[0]-1), 'bin_upper_edge']
                
                # Set group reduction lvl count of total sum
                df_temp_copy.loc[df_row[0], 'bin_edge_lag1']=sclr_bin_edge

            # Update bin edges: We need to keep track of start and end points, and also if the values are monotonic increasing or decreasing
            elif (bool_pre==False) & (df_row[0] > 0):
                
                # New update edge, looking back one row
                sclr_bin_edge=df_temp_copy.loc[(df_row[0]-1), 'bin_lower_edge']
                    
                # Set group reduction lvl count of total sum
                df_temp_copy.loc[df_row[0], 'bin_edge_lag1']=sclr_bin_edge
                                
            # Append the row to our reduced data frame
            list_hld_df_data_reduced.append(df_temp_copy[df_temp_copy.index==df_row[0]])

            # Increment reduction level counter and set sum prct holder variable and count variable to 0
            sclr_reduction_lvl += 1
            sclr_sum_prct_tot = 0
            sclr_sum_count_tot = 0


        # We hit the last row
        if df_row[0]==len(df_temp)-1:

            # Append the row to our reduced data frame
            list_hld_df_data_reduced.append(df_temp_copy[df_temp_copy.index==df_row[0]])
            
            print ("LAST ROW: {}\n".format(df_row[0]))

    # Write back to original
    df_temp=df_temp_copy.copy()

    print ("Total elasped time is: {}s. Nr of rows going out is: {}".format(round(time.time()-tick, 2), len(df_temp)))
    
    return df_temp, list_hld_df_data_reduced
